In [ ]:
# Clone (if needed), cd into the app dir, and sanity-check files
import os, subprocess, pathlib, sys, glob

REPO_URL = "https://github.com/Soph231/Food_inflation_visualization.git"
REPO_DIR = pathlib.Path("/content/Food_inflation_visualization")
APP_DIR  = REPO_DIR / "preprocessing_mini_sample" / "app"

if not APP_DIR.exists():
    print("Cloning repository...")
    subprocess.run(["git", "clone", REPO_URL, str(REPO_DIR)], check=True)
else:
    print("Repository already present. Pulling latest...")
    subprocess.run(["git", "-C", str(REPO_DIR), "pull"], check=False)

os.chdir(APP_DIR)
print("Working directory:", os.getcwd())
print("Files in app/:", sorted(os.listdir(".")))

# Tell the later cell exactly which file to run
APP_FILE = "sample_app.py"   # <-- your filename
if not pathlib.Path(APP_FILE).exists():
    raise FileNotFoundError(f"{APP_FILE} not found in {APP_DIR}. Check the filename.")


In [ ]:
# Install the app requirements and pyngrok for tunneling
!pip -q install -r requirements.txt pyngrok

# (Optional) show versions for debugging
import dash, plotly, pandas as pd
print("dash:", dash.__version__)
print("plotly:", plotly.__version__)
print("pandas:", pd.__version__)


In [ ]:
# Start Dash (sample_app.py) and expose a public URL (ngrok if token present, else Cloudflare)
import os, sys, shlex, subprocess, time, itertools, re, stat, urllib.request

PORT = 8050

def start_dash():
    # Bind to all interfaces just in case; modify in your app if supported
    cmd = f"{sys.executable} {shlex.quote(APP_FILE)}"
    print("Launching:", cmd)
    proc = subprocess.Popen(
        shlex.split(cmd),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
    )
    # Stream a few startup logs
    time.sleep(3)
    for line in itertools.islice(iter(proc.stdout.readline, b''), 40):
        print(line.decode("utf-8").rstrip())
    return proc

def with_ngrok():
    try:
        from pyngrok import ngrok
        token = os.environ.get("NGROK_AUTHTOKEN") or os.environ.get("NGROK_TOKEN")
        if not token:
            return None
        ngrok.set_auth_token(token)
        t = ngrok.connect(PORT)
        return t.public_url
    except Exception as e:
        print("[ngrok] fallback because:", e)
        return None

def with_cloudflared():
    bin_path = "/usr/local/bin/cloudflared"
    if not os.path.exists(bin_path):
        url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"
        print("Downloading cloudflared…")
        urllib.request.urlretrieve(url, bin_path)
        os.chmod(bin_path, stat.S_IRWXU | stat.S_IRGRP | stat.S_IROTH)

    proc_cf = subprocess.Popen(
        [bin_path, "tunnel", "--url", f"http://localhost:{PORT}", "--no-autoupdate"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )

    public_url = None
    for _ in range(180):  # up to ~3 minutes
        line = proc_cf.stdout.readline().strip()
        if not line:
            time.sleep(1); continue
        print(line)
        m = re.search(r"https://[a-z0-9.-]+trycloudflare\.com", line)
        if m:
            public_url = m.group(0)
            break
    return public_url, proc_cf

# 1) Start Dash app
dash_proc = start_dash()

# 2) Try ngrok (if token set), else Cloudflare
public_url = with_ngrok()
if public_url:
    print("\n✅ Public URL (ngrok):", public_url)
else:
    public_url, cf_proc = with_cloudflared()
    if public_url:
        print("\n✅ Public URL (Cloudflare):", public_url)
    else:
        print("\n❌ Could not obtain a public URL. See logs above.")
        print("   Tip: set NGROK_AUTHTOKEN env var, or ensure cloudflared printed a trycloudflare.com URL.")